In [ ]:
!pip install -U lightautoml

In [ ]:
!pip install --upgrade torch

In [ ]:
!pip install textstat

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
# Работа с файловой системой и HTTP-запросами
import os
import requests

# Библиотеки для работы с данными и моделями
import numpy as np  # Массивы и математика
import pandas as pd  # Работа с табличными данными
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report  # Метрики оценки моделей
from sklearn.model_selection import train_test_split  # Разделение данных на обучающую и тестовую выборки
from sklearn.preprocessing import LabelEncoder  # Кодирование категориальных признаков

# Работа с нейронными сетями
import torch  # Фреймворк для глубокого обучения
from transformers import AutoTokenizer, AutoModel  # Токенизатор и модель из библиотеки Transformers

# Работа с текстами
import textstat  # Лингвистические метрики текстов
import re  # Регулярные выражения

# Прогресс-бар для циклов
from tqdm import tqdm  # Удобный прогресс-бар

# Работа с временем и датами
from datetime import datetime  # Работа с временными метками

# LightAutoML для автоматического машинного обучения
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML  # Предустановки моделей
from lightautoml.tasks import Task  # Определение задачи (например, классификация, регрессия)
from lightautoml.report.report_deco import ReportDeco, ReportDecoUtilized  # Генерация отчетов
from lightautoml.addons.tabular_interpretation import SSWARM  # Интерпретация моделей

# typing для аннотаций типов
from typing import List  # Аннотация списка


In [5]:
# Инициализация токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained("sergeyzh/rubert-tiny-turbo")
model = AutoModel.from_pretrained("sergeyzh/rubert-tiny-turbo")

# Функция для предобработки текста (пример реализации)
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    return text.lower()

# Функция для извлечения первой даты из текста
def extract_first_date(text):
    if not isinstance(text, str):
        return None

    date_pattern = r'\b\d{1,2}[./-]\d{1,2}[./-]\d{2,4}\b'
    match = re.search(date_pattern, text)

    if match:
        date_str = match.group()
        for fmt in ("%d.%m.%Y", "%d/%m/%Y", "%d-%m-%Y", "%d.%m.%y", "%d/%m/%y", "%d-%m-%y"):
            try:
                return datetime.strptime(date_str, fmt)
            except ValueError:
                continue

    return None

# Функция для вычисления разницы в днях между двумя датами
def calculate_days_difference(extracted_date, reference_date):
    if pd.isna(extracted_date) or pd.isna(reference_date):
        return None
    return (reference_date - extracted_date).days

# Функция для получения эмбеддингов из текста
def get_embeddings(text: str):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Функция для генерации текстовых признаков с использованием textstat
def get_textstat_features(text):
    features = {}
    if not isinstance(text, str):
        return features
    # Расчет признаков textstat
    features['flesch_reading_ease'] = textstat.flesch_reading_ease(text)
    features['smog_index'] = textstat.smog_index(text)
    features['flesch_kincaid_grade'] = textstat.flesch_kincaid_grade(text)
    features['coleman_liau_index'] = textstat.coleman_liau_index(text)
    features['automated_readability_index'] = textstat.automated_readability_index(text)
    features['dale_chall_readability_score'] = textstat.dale_chall_readability_score(text)
    features['difficult_words'] = textstat.difficult_words(text)
    features['linsear_write_formula'] = textstat.linsear_write_formula(text)
    features['gunning_fog'] = textstat.gunning_fog(text)
    features['text_standard'] = textstat.text_standard(text, float_output=True)
    features['syllable_count'] = textstat.syllable_count(text)
    features['lexicon_count'] = textstat.lexicon_count(text)
    features['sentence_count'] = textstat.sentence_count(text)
    return features

# Функция для извлечения признаков из DataFrame
def extract_features(df, text_column):
    # Предобработка текста
    df[text_column] = df[text_column].apply(preprocess_text)

    # Извлечение номеров контрактов и их замена в тексте
    def replace_contract_numbers(text):
        if not isinstance(text, str):
            return text, False
        contract_pattern = r'(\b\d{2,}-\d{4,}\b)'
        match = re.search(contract_pattern, text)
        if match:
            return re.sub(contract_pattern, '[CONTRACT_NUMBER]', text), True
        return text, False

    df[text_column], has_contract_number = zip(
        *df[text_column].apply(replace_contract_numbers))

    # Извлечение дат и их замена в тексте
    def replace_dates_and_extract(text):
        if not isinstance(text, str):
            return text, None
        date_pattern = r'\b\d{1,2}[./-]\d{1,2}[./-]\d{2,4}\b'
        match = re.search(date_pattern, text)
        if match:
            extracted_date = extract_first_date(match.group())
            text = re.sub(date_pattern, '[DATE]', text)
            return text, extracted_date
        return text, None

    df[text_column], extracted_dates = zip(
        *df[text_column].apply(replace_dates_and_extract))

    # Вычисление разницы в датах, если существует столбец 'Date'
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
        df['date_difference'] = [
            calculate_days_difference(extracted_date, reference_date)
            for extracted_date, reference_date in zip(extracted_dates, df['Date'])
        ]
    else:
        df['date_difference'] = None

    # Получение эмбеддингов
    embeddings = df[text_column].progress_apply(get_embeddings)

    # Развертывание эмбеддингов в индивидуальные столбцы
    embeddings_df = pd.DataFrame(embeddings.tolist(), index=df.index)
    embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

    # Подсчет количества слов в тексте
    word_count = df[text_column].apply(lambda x: len(x.split()))

    # Получение признаков textstat
    textstat_features = df[text_column].apply(get_textstat_features)
    textstat_df = pd.DataFrame(textstat_features.tolist())

    # Объединение признаков обратно в исходный DataFrame
    df = pd.concat([df.reset_index(drop=True), pd.DataFrame({
        'has_contract_number': has_contract_number,
        'word_count': word_count
    }), embeddings_df.reset_index(drop=True), textstat_df.reset_index(drop=True)], axis=1)
    df = df.drop(columns=[text_column])
    return df

# Функция для загрузки и предобработки данных
def load_data(file_path, sep='\t', header=None, names=None):
    data = pd.read_csv(file_path, sep=sep, header=header, names=names)
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'], format='%d.%m.%Y')
    data['Price'] = data['Price'].replace({',': '.', '-': '.'}, regex=True)
    data['Price'] = pd.to_numeric(data['Price'], errors='coerce')
    data['Price'] = data['Price'].apply(lambda x: float(x))
    return data


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

In [7]:
# пути к файлам
train_file_path = '/content/drive/MyDrive/biv hack/for-teams/train_dataset.csv'
test_file_path = '/content/drive/MyDrive/biv hack/for-teams/payments_training.tsv'


In [10]:
# Загрузка обучающих данных
train = load_data(train_file_path, sep=',', header=0, names=['ID', 'Content', 'reasoning', 'TARGET', 'Date', 'Price', 'Fold'])
train = train[['Content', 'Date', 'TARGET', 'Price']]

# Загрузка тестовых данных
test = load_data(test_file_path, sep='\t', header=None, names=['ID', 'Date', 'Price', 'Content', 'TARGET'])
test = test[['Content', 'Date', 'TARGET', 'Price']]

# Инициализация progress_apply
tqdm.pandas()

# Извлечение признаков из обучающих и тестовых данных
train = extract_features(train, 'Content')
test = extract_features(test, 'Content')

100%|██████████| 500/500 [00:07<00:00, 63.90it/s]


In [14]:
full_train = pd.concat([train, test], ignore_index=True)

,Date,TARGET,Price,date_difference,has_contract_number,word_count,embedding_0,embedding_1,embedding_2,embedding_3,...,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard,syllable_count,lexicon_count,sentence_count
0,2024-11-07,SERVICE,32600.0,76.0,False,8,-0.014197,-0.113569,-0.311699,-0.756214,...,8.66,7.9,19.82,0,3.0,3.20,0.0,8,8,1
1,2024-11-07,SERVICE,30900.0,-121.0,False,6,-0.268744,-0.247321,0.181615,-0.624568,...,9.15,9.8,22.36,0,2.0,2.40,10.0,6,6,1
2,2024-11-07,FOOD_GOODS,4210.0,NaN,False,14,0.404704,0.311802,0.141604,-0.036590,...,16.54,12.7,23.16,0,2.5,2.80,3.0,14,14,2
3,2024-11-07,TAX,4630.0,NaN,False,2,-0.004122,0.046135,-0.452642,-0.301388,...,33.20,31.4,19.53,0,0.0,0.80,0.0,2,2,1
4,2024-11-07,SERVICE,8000.0,316.0,False,7,-0.482871,-0.159998,0.082481,-0.421207,...,6.56,7.0,19.77,0,2.5,2.80,7.0,7,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6465,2024-11-07,TAX,2610.0,NaN,False,2,0.107667,0.274753,-0.246876,-0.057394,...,7.10,10.2,19.53,0,0.0,0.80,0.0,2,2,1
6466,2024-11-07,BANK_SERVICE,31200.0,NaN,False,15,-0.277803,0.602821,-0.585012,-0.808449,...,21.57,18.1,20.17,0,6.5,6.00,0.0,15,15,1
6467,2024-11-07,SERVICE,18200.0,197.0,False,7,0.053888,0.204427,-0.376022,-0.771425,...,9.87,9.7,22.03,0,2.5,2.80,10.0,7,7,1
6468,2024-11-07,FOOD_GOODS,2870000.0,NaN,False,19,0.116363,0.089819,-0.153850,-0.324475,...,6.43,8.2,22.94,0,0.9,1.52,23.0,19,19,5


In [15]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 3600
TARGET_NAME = 'TARGET'

In [16]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [29]:
task = Task('multiclass', metric='auc_mu')

In [30]:
roles = {
    'target': TARGET_NAME,
    'drop': ['Date']
}

In [31]:
automl = TabularUtilizedAutoML(
    task = task,
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
)

In [32]:
out_of_fold_predictions = automl.fit_predict(full_train, roles = roles, verbose = 1)

[21:23:03] Start automl utilizator with listed constraints:


INFO:lightautoml.addons.utilization.utilization:Start automl utilizator with listed constraints:


[21:23:03] - time: 3600.00 seconds


INFO:lightautoml.addons.utilization.utilization:- time: 3600.00 seconds


[21:23:03] - CPU: 4 cores


...

INFO:lightautoml.automl.blend:Blending: iteration 0: score = 0.9999708472263775, weights = [0.76295125 0.23704872]


[22:10:39] Blending: iteration 1: score = 0.9999708472263775, weights = [0.76295125 0.23704872]


INFO:lightautoml.automl.blend:Blending: iteration 1: score = 0.9999708472263775, weights = [0.76295125 0.23704872]


[22:10:39] Blending: no score update. Terminated



INFO:lightautoml.automl.blend:Blending: no score update. Terminated



In [35]:
test_predictions = automl.predict(test)

In [48]:
class_labels = automl.outer_pipes[0].ml_algos[0].models[0][0].reader.class_mapping
label_class = {ind:class_ for class_, ind in class_labels.items()}

# Переводим вероятности в строковые значения классов
predicted_indices = np.argmax(test_predictions.data, axis=1)
predicted_classes = [label_class[idx] for idx in predicted_indices]

In [51]:
from sklearn.metrics import precision_score

In [ ]:
precision_score(test['TARGET'].values, predicted_classes, average='weighted')

In [ ]:
accuracy = np.mean(test['TARGET'].values == predicted_classes)

# Сохранение модели

In [34]:
with open('automl_model.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [ ]:
with open('automl_model.pkl', 'rb') as f:
    loaded_automl = pickle.load(f)

In [ ]:
test_predictions = loaded_automl.predict(test_final)

In [ ]:
class_labels = loaded_automl.reader.class_mapping
label_class = {ind:class_ for class_, ind in class_labels.items()}

# Переводим вероятности в строковые значения классов
predicted_indices = np.argmax(test_predictions.data, axis=1)
predicted_classes = [label_class[idx] for idx in predicted_indices]